In [ ]:
import requests
import base64
import pandas as pd
import time

In [ ]:
# Spotify API credentials
client_id = ''
client_secret = ''

In [ ]:
def get_track_info(df, client_id, client_secret, max_retries=5, initial_delay=1):
    client_creds = f"{client_id}:{client_secret}"
    client_creds_b64 = base64.b64encode(client_creds.encode()).decode()

    token_url = "https://accounts.spotify.com/api/token"
    token_data = {"grant_type": "client_credentials"}
    token_headers = {"Authorization": f"Basic {client_creds_b64}"}

    r = requests.post(token_url, data=token_data, headers=token_headers)

    if r.status_code != 200:
        print(f"Failed to get token: {r.status_code} - {r.text}")
        return None

    token_response_data = r.json()
    access_token = token_response_data.get('access_token')

    if not access_token:
        print("Failed to retrieve access token.")
        return None

    headers = {"Authorization": f"Bearer {access_token}"}
    track_details = []

    for spotify_id in df['spotify_id']:
        retries = 0
        delay = initial_delay

        while retries < max_retries:
            track_url = f"https://api.spotify.com/v1/tracks/{spotify_id}"
            track_response = requests.get(track_url, headers=headers)

            if track_response.status_code == 200:
                track_data = track_response.json()

                audio_features_url = f"https://api.spotify.com/v1/audio-features/{spotify_id}"
                audio_features_response = requests.get(audio_features_url, headers=headers)

                if audio_features_response.status_code == 200:
                    audio_features_data = audio_features_response.json()

                    track_details.append({
                        'track_name': track_data.get('name'),
                        'spotify_id': spotify_id,
                        'artist_name': track_data['artists'][0].get('name'),
                        'album': track_data['album'].get('name'),
                        'release_date': track_data['album'].get('release_date'),
                        'preview_url': track_data.get('preview_url'),
                        'audio_features': audio_features_data
                    })
                    time.sleep(initial_delay)  # Introduce delay between successful requests
                    break

                elif audio_features_response.status_code == 429:
                    retry_after = int(audio_features_response.headers.get('Retry-After', delay))
                    print(f"Rate limit exceeded for {spotify_id} (audio features), retrying after {retry_after} seconds...")
                    time.sleep(retry_after)
                    retries += 1
                    delay *= 2  # Exponential backoff
                else:
                    print(f"Failed to get audio features for {spotify_id}: {audio_features_response.status_code} - {audio_features_response.text}")
                    break

            elif track_response.status_code == 429:
                retry_after = int(track_response.headers.get('Retry-After', delay))
                print(f"Rate limit exceeded for {spotify_id}, retrying after {retry_after} seconds...")
                time.sleep(retry_after)
                retries += 1
                delay *= 2  # Exponential backoff
            else:
                print(f"Failed to get track info for {spotify_id}: {track_response.status_code} - {track_response.text}")
                break

    df_details = pd.DataFrame(track_details)
    return df_details


In [ ]:
song_df = pd.read_csv('/song_df.csv')

In [ ]:
song_df_with_audio_features = get_track_info(song_df, client_id, client_secret)

In [ ]:
song_df_with_audio_features[22365:22366]

,Unnamed: 0,track_id,name,artist,spotify_preview_url,spotify_id,tags,genre,year,duration_ms,...,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
22365,22365,TRBGFKW128F427FCBE,Tight,Mindless Self Indulgence,https://p.scdn.co/mp3-preview/563278b9ab72908e...,0fqDLp5s0wctxSXt35uNXi,"punk, industrial",NaN,2011,169346,...,5,-5.391,1,0.242,0.015,0.000402,0.318,0.447,153.689,4


In [ ]:
song_df_with_audio_features.to_csv('/song_df_with_audio_features.csv')